# RPLib Problem 001
## March Madness Dataset

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import copy
import os
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.stats import skew
from tqdm import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import itertools
import joblib

In [3]:
from pathlib import Path
home = str(Path.home())
home

'/home/jupyter-pander14'

In [4]:
import sys
sys.path.insert(0,"%s/rankability_toolbox_dev"%home)
import pyrankability

In [5]:
sys.path.insert(0,"%s/sensitivity_study/src"%home)
from sensitivity_tests import *
from utilities import *
from base import *

In [6]:
games={}
remaining_games={}
madness_teams={}
all_teams={}
years = ["2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"]
for year in years:
    games[year],remaining_games[year] = read_data(f'{home}/marchmadness_study/data/%steams.txt'%year,f'{home}/marchmadness_study/data/%sgames.txt'%year,f'{home}/marchmadness_study/data/%sMadnessTeams.txt'%year)
    madness_teams[year] = list(np.unique(list(games[year].team1_name.loc[games[year].team1_madness == 1]) + list(games[year].team2_name.loc[games[year].team2_madness == 1])))
    all_teams[year] = list(np.unique(list(games[year].team1_name) + list(games[year].team2_name)))
print(year)
games[year]

2018


,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
0,1,16,737011,2017-11-13,1,83,-1,69,Arkansas_St,Abilene_Chr,0,0
1,1,41,737114,2018-02-24,-1,74,1,72,Cent_Arkansas,Abilene_Chr,0,0
3,1,143,737018,2017-11-20,-1,75,1,67,Lipscomb,Abilene_Chr,1,0
4,1,143,737045,2017-12-17,1,67,-1,65,Lipscomb,Abilene_Chr,1,0
5,1,199,737056,2017-12-28,1,77,-1,74,New_Orleans,Abilene_Chr,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5535,351,318,737048,2017-12-20,1,91,-1,74,Utah_St,Youngstown_St,0,0
5536,351,339,737086,2018-01-27,1,85,-1,67,WI_Green_Bay,Youngstown_St,0,0
5537,351,340,737084,2018-01-25,1,66,-1,55,WI_Milwaukee,Youngstown_St,0,0
5538,351,347,737074,2018-01-15,1,77,-1,67,Wright_St,Youngstown_St,1,0


In [7]:
remaining_games[year]

,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
2,1,73,737130,2018-03-12,1,80,-1,73,Drake,Abilene_Chr,0,0
32,2,313,737125,2018-03-07,1,97,-1,90,UNLV,Air_Force,0,0
42,3,79,737126,2018-03-08,0,67,0,58,E_Michigan,Akron,0,0
59,4,137,737128,2018-03-10,0,86,0,63,Kentucky,Alabama,1,1
68,4,326,737135,2018-03-17,0,81,0,58,Villanova,Alabama,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5478,347,287,737133,2018-03-15,0,73,0,47,Tennessee,Wright_St,1,1
5491,348,197,737126,2018-03-08,0,85,0,75,New_Mexico,Wyoming,0,0
5496,349,92,737136,2018-03-18,0,75,0,70,Florida_St,Xavier,1,1
5498,349,234,737127,2018-03-09,0,75,0,72,Providence,Xavier,1,1


In [124]:
# Note to future self: Parameters from FODS paper but might need to be optimized
direct_thress = [0,3.]
spread_thress = [0]
weight_indirects = [0]
domains_ranges = [('all','madness')]

# fracs represent how much of the data to include
fracs = [0.5,0.6,0.7,0.8,0.9,1.]

In [125]:
massey_rankings = {}
colley_rankings = {}
massey_rs = {}
colley_rs = {}
colley_perms = {}
massey_perms = {}

outer_keys = list(itertools.product(domains_ranges,years))
for domain_range,year in tqdm(outer_keys):
    # set the team_domain
    team_domain = None
    if domain_range[0] == 'madness':
        team_domain = madness_teams[year]
    elif domain_range[0] == 'all':
        team_domain = all_teams[year]

    # set the team_range
    team_range = None
    if domain_range[1] == 'madness':
        team_range = madness_teams[year]
    elif domain_range[1] == 'all':
        team_range = all_teams[year]

    columns = ["frac","direct_thres","spread_thres","weight_indirect"]+team_range
    massey_rankings[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_rankings[(domain_range,year)] = pd.DataFrame(columns=columns)
    massey_rs[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_rs[(domain_range,year)] = pd.DataFrame(columns=columns)
    massey_perms[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_perms[(domain_range,year)] = pd.DataFrame(columns=columns)

    game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                            "team1_score":games[year]['points1'],
                            "team1_H_A_N": games[year]['H_A_N1'],
                            "team2_name":games[year]['team2_name'],
                            "team2_score":games[year]['points2'],
                            "team2_H_A_N": games[year]['H_A_N1'],
                            "date": games[year]['date']
                           }).sort_values(by='date').drop('date',axis=1)
    mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
    game_df = game_df.loc[mask]

    keys = list(itertools.product(fracs,direct_thress,spread_thress,weight_indirects))

    def compute(frac,direct_thres,spread_thres,weight_indirect,team_range,all_teams,game_df):
        upper = int(len(game_df)*frac)
        game_df_sample = game_df.iloc[:upper,:]

        map_func = lambda linked: pyrankability.construct.colley_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
        colley_matrix,colley_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
        colley_matrix = colley_matrix.reindex(index=all_teams,columns=all_teams)
        #import pdb; pdb.set_trace()
        colley_b = colley_b.reindex(all_teams)
        #mask = colley_b.isna()
        #colley_b = colley_b.loc[~mask]
        #colley_matrix = colley_matrix.loc[~mask,~mask]
        inxs = []
        for team in team_range:
            inxs.append(int(np.where(colley_b.index == team)[0][0]))
        #inxs = list(np.where(~colley_b.index.isin(team_range))[0]) #list(np.where(mask)[0])
        ranking1,r1,perm1 = pyrankability.rank.ranking_from_matrices(colley_matrix.fillna(0),colley_b.fillna(0),np.array(inxs))
        
        map_func = lambda linked: pyrankability.construct.massey_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
        massey_matrix,massey_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
        massey_matrix = massey_matrix.reindex(index=all_teams,columns=all_teams)
        massey_b = massey_b.reindex(all_teams)
        #mask = massey_b.isna()
        #massey_b = massey_b.loc[~mask]
        #massey_matrix = massey_matrix.loc[~mask,~mask]    
        #inxs = list(np.where(mask)[0])    
        #inxs = list(np.where(~massey_b.index.isin(team_range))[0]) #list(np.where(mask)[0])
        inxs = []
        for team in team_range:
            inxs.append(int(np.where(massey_b.index == team)[0][0]))
        ranking2,r2,perm2 = pyrankability.rank.ranking_from_matrices(massey_matrix.fillna(0),massey_b.fillna(0),np.array(inxs))
        
        ranking_values1 = [frac,direct_thres,spread_thres,weight_indirect]+list(ranking1)
        ranking_values2 = [frac,direct_thres,spread_thres,weight_indirect]+list(ranking2)
        r_values1 = [frac,direct_thres,spread_thres,weight_indirect]+list(r1)
        r_values2 = [frac,direct_thres,spread_thres,weight_indirect]+list(r2)
        perm_values1 = [frac,direct_thres,spread_thres,weight_indirect]+list(perm1)
        perm_values2 = [frac,direct_thres,spread_thres,weight_indirect]+list(perm2)
        
        return (pd.Series(ranking_values1,index=columns),
                pd.Series(ranking_values2,index=columns),
                pd.Series(r_values1,index=columns),
                pd.Series(r_values2,index=columns),
                pd.Series(perm_values1,index=columns),
                pd.Series(perm_values2,index=columns)              
               )

    #frac,direct_thres,spread_thres,weight_indirect = keys[0]
    #for frac,direct_thres,spread_thres,weight_indirect in keys:
    #    compute(frac,direct_thres,spread_thres,weight_indirect,team_range,all_teams[year],game_df)
    results = Parallel(n_jobs=-1)(delayed(compute)(frac,direct_thres,spread_thres,weight_indirect,team_range,all_teams[year],game_df) for frac,direct_thres,spread_thres,weight_indirect in keys)

    c = 0
    for i,key in enumerate(keys):
        frac,direct_thres,spread_thres,weight_indirect = key
        colley,massey,colley_r,massey_r,colley_perm,massey_perm = results[i]
        massey.name = c
        colley.name = c
        colley_r.name=c
        massey_r.name=c
        colley_perm.name=c
        massey_perm.name=c
        massey_rankings[(domain_range,year)] = massey_rankings[(domain_range,year)].append(massey)
        colley_rankings[(domain_range,year)] = colley_rankings[(domain_range,year)].append(colley)
        massey_rs[(domain_range,year)] = massey_rs[(domain_range,year)].append(massey_r)
        colley_rs[(domain_range,year)] = colley_rs[(domain_range,year)].append(colley_r)
        massey_perms[(domain_range,year)] = massey_perms[(domain_range,year)].append(massey_perm)
        colley_perms[(domain_range,year)] = colley_perms[(domain_range,year)].append(colley_perm)
        c+=1


100%|██████████| 17/17 [01:54<00:00,  7.11s/it]

## What parameters should we pick?
One way to focus our analysis is to study the parameters that result in the best accuracy when predicting future games.

In [128]:
def calc_predictability(games,r):
    numberCorrectPredictions = 0
    numGames = 0
    for i in games.index:
        team1ID = games.loc[i, "team1_name"]
        team1Score = games.loc[i, "points1"]
        team2ID = games.loc[i, "team2_name"]
        team2Score = games.loc[i, "points2"]
        
        if team1ID in r.index and team2ID in r.index:
            if team1Score > team2Score and r.loc[team1ID] > r.loc[team2ID]:
                numberCorrectPredictions += 1
            elif team2Score > team1Score and r.loc[team2ID] > r.loc[team1ID]:
                numberCorrectPredictions += 1
            elif team1Score == team2Score and r.loc[team1ID] == r.loc[team2ID]:
                numberCorrectPredictions += 1

            numGames += 1
           
    predictability = numberCorrectPredictions/numGames*100
    return predictability,numGames

In [129]:
pred_df = pd.DataFrame(columns=['frac','domain','range',"direct_thres","spread_thres","weight_indirect",'Method','Year','Predictability','rankings'])

#frac = 1. # Use all the data available

keys = list(itertools.product(fracs,domains_ranges,direct_thress,spread_thress,weight_indirects,years))

c=0
for frac,domain_range,dt,st,iw,year in keys:
    dom = domain_range[0]
    ran = domain_range[1]
    method = 'Massey'
    r = massey_rs[(domain_range,year)].set_index(["frac","direct_thres","spread_thres","weight_indirect"]).loc[(frac,dt,st,iw)]
    rankings = massey_rankings[(domain_range,year)].set_index(["frac","direct_thres","spread_thres","weight_indirect"]).loc[(frac,dt,st,iw)]
    val = calc_predictability(remaining_games[year],r)[0]
    entry = pd.Series([frac,dom,ran,dt,st,iw,method,year,val,rankings],name=c,index=pred_df.columns)
    c+=1
    pred_df=pred_df.append(entry)

    method = 'Colley'
    r = colley_rs[(domain_range,year)].set_index(["frac","direct_thres","spread_thres","weight_indirect"]).loc[(frac,dt,st,iw)]
    rankings = colley_rankings[(domain_range,year)].set_index(["frac","direct_thres","spread_thres","weight_indirect"]).loc[(frac,dt,st,iw)]
    val = calc_predictability(remaining_games[year],r)[0]
    entry = pd.Series([frac,dom,ran,dt,st,iw,method,year,val,rankings],name=c,index=pred_df.columns)
    c+=1
    pred_df=pred_df.append(entry)
    

In [130]:
pred_df

,frac,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,Predictability,rankings
0,0.5,all,madness,0,0,0,Massey,2002,67.415730,Alabama 14.0 Alcorn_St 65.0 ...
1,0.5,all,madness,0,0,0,Colley,2002,64.044944,Alabama 20.0 Alcorn_St 62.0 ...
2,0.5,all,madness,0,0,0,Massey,2003,65.476190,Alabama 23.0 Arizona 3.0 Arizon...
3,0.5,all,madness,0,0,0,Colley,2003,55.952381,Alabama 13.0 Arizona 2.0 Arizon...
4,0.5,all,madness,0,0,0,Massey,2004,63.440860,Air_Force 42.0 Alabama 23.0 Alabam...
...,...,...,...,...,...,...,...,...,...,...
403,1.0,all,madness,3,0,0,Colley,2016,73.809524,Akron 30.0 Arizona 17....
404,1.0,all,madness,3,0,0,Massey,2017,54.838710,Arizona 26.0 Arkansas 38.0 Ba...
405,1.0,all,madness,3,0,0,Colley,2017,51.612903,Arizona 2.0 Arkansas 26.0 Ba...
406,1.0,all,madness,3,0,0,Massey,2018,64.044944,Alabama 40.0 Arizona 26.0 Ar...


In [131]:
scores = pred_df.groupby(['Method','frac','domain','range','direct_thres','spread_thres','weight_indirect'])['Predictability'].median().reset_index()
scores.sort_values(by='Predictability')

,Method,frac,domain,range,direct_thres,spread_thres,weight_indirect,Predictability
0,Colley,0.5,all,madness,0.0,0,0,62.637363
5,Colley,0.7,all,madness,3.0,0,0,62.921348
7,Colley,0.8,all,madness,3.0,0,0,62.921348
1,Colley,0.5,all,madness,3.0,0,0,63.095238
13,Massey,0.5,all,madness,3.0,0,0,63.095238
2,Colley,0.6,all,madness,0.0,0,0,63.333333
3,Colley,0.6,all,madness,3.0,0,0,63.333333
12,Massey,0.5,all,madness,0.0,0,0,63.917526
16,Massey,0.7,all,madness,0.0,0,0,64.285714
9,Colley,0.9,all,madness,3.0,0,0,64.285714


In [132]:
inx_values = ['domain','range','direct_thres','spread_thres','weight_indirect']
best_df = scores.groupby(['frac','Method']).apply(lambda df: pd.Series(df.set_index(inx_values).idxmax()[0],index=inx_values)).reset_index()
best_df

,frac,Method,domain,range,direct_thres,spread_thres,weight_indirect
0,0.5,Colley,all,madness,3.0,0,0
1,0.5,Massey,all,madness,0.0,0,0
2,0.6,Colley,all,madness,0.0,0,0
3,0.6,Massey,all,madness,0.0,0,0
4,0.7,Colley,all,madness,0.0,0,0
5,0.7,Massey,all,madness,3.0,0,0
6,0.8,Colley,all,madness,0.0,0,0
7,0.8,Massey,all,madness,3.0,0,0
8,0.9,Colley,all,madness,0.0,0,0
9,0.9,Massey,all,madness,3.0,0,0


In [133]:
import altair as alt

graph_df = best_df.set_index(inx_values+["frac","Method"]).join(scores.set_index(inx_values+["frac","Method"])).reset_index().sort_values(by='frac')

alt.Chart(graph_df).mark_line().encode(
    x='frac',
    y=alt.Y('Predictability',scale=alt.Scale(domain=[60, 70])),
    color='Method:N'
)

alt.Chart(...)

In [134]:
best_pred_df = pred_df.set_index(inx_values+["frac","Method"]).loc[best_df.set_index(inx_values2+["frac","Method"]).index]
best_pred_df

Year  \
domain range   direct_thres spread_thres weight_indirect frac Method         
all    madness 3.0          0            0               0.5  Colley  2002   
                                                              Colley  2003   
                                                              Colley  2004   
                                                              Colley  2005   
                                                              Colley  2006   
...                                                                    ...   
               0.0          0            0               1.0  Massey  2014   
                                                              Massey  2015   
                                                              Massey  2016   
                                                              Massey  2017   
                                                              Massey  2018   

                                                                      Predictability  \
domain range   direct_thres spread_thres weight_indirect frac Method                   
all    madness 3.0          0            0               0.5  Colley       64.044944   
                                                              Colley       67.857143   
                                                              Colley       56.989247   
                                                              Colley       65.934066   
                                                              Colley       63.953488   
...                                                                              ...   
               0.0          0            0               1.0  Massey       65.000000   
                                                              Massey       69.387755   
                                                              Massey       70.238095   
                                                              Massey       54.838710   
                                                              Massey       65.168539   

                                                                                                               rankings  
domain range   direct_thres spread_thres weight_indirect frac Method                                                     
all    madness 3.0          0            0               0.5  Colley  Alabama           32.0
Alcorn_St         64.0
...  
                                                              Colley  Alabama         7.0
Arizona         3.0
Arizon...  
                                                              Colley  Air_Force      36.0
Alabama        22.0
Alabam...  
                                                              Colley  Alabama           26.0
Alabama_A&M       65.0
...  
                                                              Colley  Air_Force        19.0
Alabama          47.0
Al...  
...                                                                                                                 ...  
               0.0          0            0               1.0  Massey  Albany_NY        64.0
American_Univ    59.0
Ar...  
                                                              Massey  Albany_NY     61.0
Arizona        3.0
Arkansas...  
                                                              Massey  Akron              53.0
Arizona            12....  
                                                              Massey  Arizona          26.0
Arkansas         35.0
Ba...  
                                                              Massey  Alabama          38.0
Arizona          24.0
Ar...  

[204 rows x 3 columns]

In [135]:
best_pred_df.reset_index().set_index('frac').loc[0.5]

,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,Predictability,rankings
frac,,,,,,,,,
0.5,all,madness,3.0,0,0,Colley,2002,64.044944,Alabama 32.0 Alcorn_St 64.0 ...
0.5,all,madness,3.0,0,0,Colley,2003,67.857143,Alabama 7.0 Arizona 3.0 Arizon...
0.5,all,madness,3.0,0,0,Colley,2004,56.989247,Air_Force 36.0 Alabama 22.0 Alabam...
0.5,all,madness,3.0,0,0,Colley,2005,65.934066,Alabama 26.0 Alabama_A&M 65.0 ...
0.5,all,madness,3.0,0,0,Colley,2006,63.953488,Air_Force 19.0 Alabama 47.0 Al...
0.5,all,madness,3.0,0,0,Colley,2007,70.238095,Albany_NY 58.0 Arizona 3.0 Arkansas ...
0.5,all,madness,3.0,0,0,Colley,2008,62.921348,American_Univ 57.0 Arizona 22.0 Ar...
0.5,all,madness,3.0,0,0,Colley,2009,61.111111,Akron 52.0 Alabama_St 63.0 Am...
0.5,all,madness,3.0,0,0,Colley,2010,58.888889,Ark_Pine_Bluff 65.0 BYU 5.0 ...


In [136]:
top_k = 15
feature_name = f'top{top_k}_intersection'
ms = pd.DataFrame(columns=['frac1','frac2','domain','range',"direct_thres","spread_thres","weight_indirect",'Method','Year',feature_name])

#keys = list(itertools.product(domains_ranges,direct_thress,spread_thress,weight_indirects,years))
pair_fracs = [sorted(fracs) for fracs in list(itertools.combinations(fracs,2))]

c=0
for index,row in best_df.iterrows():
    dom,ran,dt,st,iw = row.loc['domain'],row.loc['range'],row.loc['direct_thres'],row.loc['spread_thres'],row.loc['weight_indirect']
    method = row.loc['Method']
    for year in years:
        for frac1,frac2 in pair_fracs:
            if method == 'Massey':
                rankings = massey_rankings[(domain_range,year)].set_index(["direct_thres","spread_thres","weight_indirect"]).loc[(dt,st,iw)]
            elif method == 'Colley':
                rankings = colley_rankings[(domain_range,year)].set_index(["direct_thres","spread_thres","weight_indirect"]).loc[(dt,st,iw)]
                #import pdb; pdb.set_trace()
            else:
                raise Exception('Unsupported')
            rankings1 = rankings.set_index('frac').loc[frac1].T
            rankings1 = rankings1.loc[rankings1 < top_k]
            rankings2 = rankings.set_index('frac').loc[frac2].T
            rankings2 = rankings2.loc[rankings2 < top_k]
            #import pdb; pdb.set_trace()
            val = len(set(rankings1.index).intersection(set(rankings2.index)))/top_k
            entry = pd.Series([frac1,frac2,dom,ran,dt,st,iw,method,year,val],name=c,index=ms.columns)
            c+=1
            ms=ms.append(entry)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/use

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/use

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/use

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/use

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
/opt/tljh/use

...

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: i

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: i

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: i

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: i

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: i

In [137]:
ms.sort_values(by=feature_name)

,frac1,frac2,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,top15_intersection
1173,0.5,0.9,all,madness,0.0,0,0,Colley,2012,0.466667
1037,0.5,0.8,all,madness,0.0,0,0,Colley,2003,0.466667
543,0.5,0.9,all,madness,0.0,0,0,Colley,2004,0.466667
1683,0.5,0.9,all,madness,0.0,0,0,Colley,2012,0.466667
1547,0.5,0.8,all,madness,0.0,0,0,Colley,2003,0.466667
...,...,...,...,...,...,...,...,...,...,...
1964,0.9,1.0,all,madness,3.0,0,0,Massey,2013,0.933333
989,0.9,1.0,all,madness,0.0,0,0,Massey,2016,0.933333
1970,0.6,0.7,all,madness,3.0,0,0,Massey,2014,0.933333
1940,0.6,0.7,all,madness,3.0,0,0,Massey,2012,0.933333


In [138]:
graph_df = ms.copy().reset_index()
graph_df["Interval"] = graph_df["frac1"].astype(str) +"-"+graph_df["frac2"].astype(str)
graph_df["Interval Width"] = np.round(-100*(graph_df["frac1"] - graph_df["frac2"]))
graph_df[f"Intersection in top {top_k}"] = graph_df[feature_name]

alt.Chart(graph_df).mark_line().encode(
    x='Year',
    y=alt.Y(f"average(Intersection in top {top_k})",scale=alt.Scale(domain=[0.5, 1.])),
    color='Interval Width:N'
).facet(column='Method')

alt.FacetChart(...)

In [139]:
import altair as alt

graph_df = ms.copy().reset_index()
graph_df["Interval"] = graph_df["frac1"].astype(str) +"-"+graph_df["frac2"].astype(str)
graph_df["Interval Width"] = np.round(-100*(graph_df["frac1"] - graph_df["frac2"]))
graph_df[f"Intersection in top {top_k}"] = graph_df[feature_name]

alt.Chart(graph_df).mark_bar().encode(
    x='Year',
    y=f"average(Intersection in top {top_k})",
).facet(row='Interval Width:N',column='Method')

alt.FacetChart(...)

In [140]:
import altair as alt

graph_df = ms.copy().reset_index()
graph_df["Interval"] = graph_df["frac1"].astype(str) +"-"+graph_df["frac2"].astype(str)
graph_df["Interval Width"] = np.round(-100*(graph_df["frac1"] - graph_df["frac2"]))
graph_df[f"Intersection in top {top_k}"] = graph_df[feature_name]

error_bars = alt.Chart(graph_df).mark_errorbar(extent='ci').encode(
    x='Interval Width:N',
    y=alt.Y(f"average(Intersection in top {top_k})",scale=alt.Scale(domain=[0.5, 1.])),
    color='Method'
)

error_bars+alt.Chart(graph_df).mark_line().encode(
    x='Interval Width:N',
    y=alt.Y(f"average(Intersection in top {top_k})",scale=alt.Scale(domain=[0.5, 1.])),
    color='Method'
)

alt.LayerChart(...)

In [141]:
sensitivity_target = ms.copy()

sensitivity_data = {}
for year in tqdm(years):
    sensitivity_data[year] = {}
    for frac in fracs:
        game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                                "team1_score":games[year]['points1'],
                                "team1_H_A_N": games[year]['H_A_N1'],
                                "team2_name":games[year]['team2_name'],
                                "team2_score":games[year]['points2'],
                                "team2_H_A_N": games[year]['H_A_N1'],
                                "date": games[year]['date']
                               }).sort_values(by='date').drop('date',axis=1)
        mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
        game_df = game_df.loc[mask]
        upper = int(len(game_df)*frac)
        game_df_sample = game_df.iloc[:upper,:]
        sensitivity_data[year][f"frac={frac}"]=game_df_sample
        
description = """
A practitioner wants to predict the degree to which a the rankings during season 
of the NCAA Men’s Basketball are likely to change as more games are played (i.e., sensitivity to more games). 
They want to start the analysis after a minimum of 50% of the games are played. 
They want to run Massey and Colley.

Sensitivity of new games will be measured as the intersection of between two 
rankings derived from before and after the new games are included.
"""
joblib.dump({'description':description,'target':sensitivity_target,'data':sensitivity_data,'other':{'madness_teams':madness_teams,'remaining_games':remaining_games,'best_df':best_df,'best_pred_df':best_pred_df,'top_k':top_k}},"/disk/RPLib/problem_0001.joblib.z")


100%|██████████| 17/17 [00:00<00:00, 53.37it/s]

['/disk/RPLib/problem_0001.joblib.z']